# How was the wheater during session in WUT?

We define exam session time as first two full weeks of Feburary (i.e. first day of session is first monday of Februray) and last two weeks of June (i.e. last day of session is last sunday of June).

## Instalation of libraries

In [24]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas

## Defining libraries and client config

In [25]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

import datetime

In [26]:
def get_special_dates(year: int) -> tuple[tuple[str, str], tuple[str, str]]:
    def get_first_full_week_of_february(year):
        # Find the first Monday of February
        february_first = datetime.date(year, 2, 1)
        first_monday = february_first + datetime.timedelta(days=(7 - february_first.weekday()) % 7)
        # Get the start and end of the first full week
        start_date = first_monday
        end_date = first_monday + datetime.timedelta(days=13)
        return start_date, end_date

    def get_last_two_full_weeks_of_june(year):
# Find the first day of July
        july_first = datetime.date(year, 7, 1)
        # Find the last Sunday that is in June
        last_sunday_in_june = july_first - datetime.timedelta(days=july_first.weekday() + 1)
        # Calculate the start of the last two full weeks
        last_week_start = last_sunday_in_june - datetime.timedelta(days=6)
        second_last_week_start = last_week_start - datetime.timedelta(days=7)
        second_last_week_end = second_last_week_start + datetime.timedelta(days=6)
        return second_last_week_start, second_last_week_end, last_week_start, last_sunday_in_june
    
    feb_start, feb_end = get_first_full_week_of_february(year)
    june_2nd_last_start, june_2nd_last_end, june_last_start, june_last_end = get_last_two_full_weeks_of_june(year)
    
    return (feb_start.strftime("%Y-%m-%d"), feb_end.strftime("%Y-%m-%d")), (june_2nd_last_start.strftime("%Y-%m-%d"), june_last_end.strftime("%Y-%m-%d"))

In [27]:
def get_weather_in_mini_between_dates(client: openmeteo_requests.Client, dates: tuple[str, str]) -> pd.DataFrame:
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": 52.22,
        "longitude": 21,
        "start_date": dates[0],
        "end_date": dates[1],
        "daily": "sunshine_duration",
        "timezone": "Europe/Berlin"
    }
    responses = client.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}
    daily_data["sunshine_duration"] = daily_sunshine_duration

    daily_dataframe = pd.DataFrame(data = daily_data)

    return pd.DataFrame(data = daily_data)

In [31]:
for i in range(2001, 2023):
    dates = get_special_dates(i)
    print("First full week of February:", dates[0])
    print("Last two full weeks of June ending in July:", dates[1])
    daily_dataframe = get_weather_in_mini_between_dates(openmeteo, dates[0])
    print(daily_dataframe)
    daily_dataframe = get_weather_in_mini_between_dates(openmeteo, dates[1])
    print(daily_dataframe)

First full week of February: ('2001-02-05', '2001-02-18')
Last two full weeks of June ending in July: ('2001-06-11', '2001-06-24')
Coordinates 52.196834564208984°N 20.922508239746094°E
Elevation 120.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s
                        date  sunshine_duration
0  2001-02-04 22:00:00+00:00           0.000000
1  2001-02-05 22:00:00+00:00        1257.033936
2  2001-02-06 22:00:00+00:00       24950.281250
3  2001-02-07 22:00:00+00:00       27503.234375
4  2001-02-08 22:00:00+00:00       29462.748047
5  2001-02-09 22:00:00+00:00          34.979782
6  2001-02-10 22:00:00+00:00       26249.193359
7  2001-02-11 22:00:00+00:00        7776.472656
8  2001-02-12 22:00:00+00:00        1052.866333
9  2001-02-13 22:00:00+00:00       29777.357422
10 2001-02-14 22:00:00+00:00         384.470428
11 2001-02-15 22:00:00+00:00       23988.042969
12 2001-02-16 22:00:00+00:00       31641.863281
13 2001-02-17 22:00:00+00:00       18000.000000
Coo

### POC